<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch8_5_pg_497~512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 감성 분석
- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방버으로 소셜미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용되고 있다. 감성 분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성 수치를 계산하는 방법을 이용한다. 
- 이러한 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되며, 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정 


> - 지도학습: 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 진행한 후 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법. 일반적인 텍스트 기반의 분류와 거의 동일

> - 비지도학습: Lexicon이라는 일종의 감성 어휘 사전을 이용. 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 갖고 있고, 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단함.-> 근데 이러면, 사실상 지도학습을 어휘 사전으로 만들어둔 것 아닌지

# 지도 학습 기반 감성 분석 실습 - IMDB 영화평


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ESAA/2023-1/필사/data/") # working directory를 설정함.

Mounted at /content/drive


In [ ]:
import pandas as pd

review_df=pd.read_csv('labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


# 피처
- id : 각 데이터의 id
- sentiment : 영화평의 Sentiment 결과 값(Target label). 1은 긍정적 평가. 0은 부정적 평가 의미.
- review : 영화평의 텍스트

# 텍스트 구성 살펴보기

In [ ]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- HTML 형식에서 추출해 < br > 태그 여전히 존재 -> 필요 없으므로 replace()를 str에 적용해 공백으로 변환
- 숫자/특수문자 필요없음 -> 정규 표현식 지원하는 re모듈 -> 영어 대/소문자가 아닌 모든 문자 찾아 공란으로 변경

In [ ]:
import re

#  html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

# 결정 값/피처 데이터 세트 생성
- 결정 값 클래스인 sentiment 칼럼을 별도로 추출 -> 결정 값 데이터 세트를 만들기 
- 원본 데이터세트에서 id와 sentiment 칼럼을 삭제해 피처 데이터 세트를 생성 
- 학습용 데이터와 테스트용 데이터 세트로 분리

In [ ]:
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test=train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
X_train.shape

(17500, 1)

# 감상평 텍스트를 피처 백터화 후 ML 분류 알고리즘을 적용해 석능 측정
- pipeline 이용
1. count 벡터화: 예측 성능 측정
2. TF-IDF 벡터화 적용
3. 분류는 로지스틱 회귀
- 이진 분류이므로, 데이터세트의 정확도와 ROC-AUC 모두 측정

In [ ]:
X_train.head()

,review
3724,T h i s v e r s i o n m o v e d a l...
23599,I r e a l l y e n j o y e d t h i s ...
11331,S a w t h i s i n t h e t h e a t e...
15745,R e c e n t l y I w a s l o o k i n g...
845,E s c a p i n g t h e l i f e o f b...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

#스톱 워드는 english, filtering, ngram은 (1,2) 로 설정해 countvectorization 수행.
#LogisticRegression의 C=10 으로 설정
pipeline=Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))])

#Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행, predict_proba()는 roc_auc 때문에 수행.
pipeline.fit(X_train['review'], y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:, 1]

print('count의 예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

ValueError: ignored

# TF-IDF 벡터화

In [ ]:
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 TF-IDF 수행.
# LogisticRegression의 C는 10으로 설정.
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred_probs)))

- 성능이 약간 더 나아짐

# 비지도학습 기반 감성 분석
- 비지도 감성 분석은 Lexicon(=감성 사전) 기반으로 긍정 감성 또는 부정 감성의 정도를 의미하는 수치(=감성 지수) 가지고 있음. (=감성지수)
- 감성 사전 구현한 대표격은 NLTK 패키지

> - NLP 패키지의 WordNet 모듈 : 시맨틱 분석을 제공하는 어휘 사전

- 시맨틱 : '문맥상 의미'
- NLP 패키지 : 시맨틱을 프로그램적으로 인터페이스할 수 있는 다양한 방법 제공
- WordNet : 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보 제공

- 각각의 품사로 구성된 개별 단어를 Synset이라는 개념을 이용해 표현
- Synset은 단순한 하나의 단어가 아니라 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념
- NLTK의 감성 사전이 아쉽게도 예측 성능 좋지 않아 실제 업무의 적용은 다른 감성 사전 적용

# 대표적인 감성 사전
- SentiWordNet : NLTK 패키지의 WordNet과 유사하게 감성 단어 전용의 WordNet을 구현한 것.
- Synset별로 감성 점수 할당(긍정/부정 감성 지수, 객관성 지수) -> 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수 계산하고 이에 기반해 결정
- VADER : 주로 소셜 미디어의 텍스트에 대한 감성 분석 제공하기 위한 패키지
뛰어난 감성 분석 결과 제공. 비교적 빠른 수행 시간 보장 -> 대용량 텍스트 데이터에 잘 사용
- Pattern : 예측 성능 측면에서 가장 주목받는 패키지

# SentiWordNet을 이용한 감성 분석

WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해
- SentiWordNet은 WordNet 기반의 synset 이용

# NLTK의 모든 데이터 세트와 패키지 내려받기

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

- WordNet의 synsets() : 파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Synset 객체 반환

In [ ]:
from nltk.corpus import wordnet as wn

term = 'present'

# 'present'라는 단어로 wordnet의 synsets 생성.
synsets = wn.synsets(term)
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 개수:', len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수: 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- synsets() 호출 시 여러 개의 Synset 객체를 가지는 리스트 반환
- synset('present.n.01') : POS 태그 ( 의미, 품사, 의미 여러가지 있어 구분하는 인덱스)
- synset : POS, 정의, 부명제 등으로 시맨틱적인 요소 표현 가능

In [ ]:
for synset in synsets:
  print('##### Synset name : ', synset.name(), '#####')
  print('POS :', synset.lexname())
  print('Definition:', synset.definition())
  print('Lemmas:', synset.lemma_names())

##### Synset name :  present.n.01 #####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
##### Synset name :  present.n.02 #####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
##### Synset name :  present.n.03 #####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
##### Synset name :  show.v.01 #####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 #####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
##### Synset name :  stage.v.01 #####
POS : verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represen

- Synset('present.n.01')과 Synset('present.n.01')는 명사지만 서로 다른 의미
- synset은 하나의 단어가 가질 수 있는 여러 가지 시맨틱 정보를 개별 클래스로 나타낸 것
- WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있다 - path_similarity() 메서드 제공

In [ ]:
# synset 객체를 단어별로 생성합니다.
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]

# 단어별 synset을 반복하면서 다른 언어의 synset과 유사도를 측정합니다.
for entity in entities:
  similarity = [round(entity.path_similarity(compared_entity),2) for compared_entity in entities]
  similarities.append(similarity)

# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame 형태로 저장합니다.
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


- SentiWordNet은 WordNet의 Synset과 유사한 senti_synsets() 모듈 : Senti_Synset 클래스를 리스트 형태로 반환

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('semti_synsets() 반환 값 개수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
semti_synsets() 반환 값 개수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


- 감성 지수(긍정/부정 감성 지수)와 객관성 지수 가지고 있음: 전혀 감성적이지 않으면 객관성 지수는 1, 감성 지수는 0

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ', fabulous.pos_score())
print('fabulous 부정감성 지수: ', fabulous.neg_score())
     

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


# SentiWordNet을 이용한 영화 감상평 감성 분석
1. 문서를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 떄 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

- SentiWordNet을 이용하기 위해서 WordNet을 이용해 문서를 다시 단어로 토큰화한 뒤 어근 추출(Lemmatization)과 품사 태깅(POS Tagging) 적용

# 품사 태깅을 수행하는 내부 함수 생성


In [ ]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

- 문서를 문장->단어 토큰->품사 태깅 후에 SentiSynset 클래스 생성하고 Polarity Score를 합산하는 함수 생성
- 각 단어의 긍정 감성 지수와 부정 감성 지수를 모두 합한 총 감성 지수가 0 이상일 경우 긍정 감성, 그렇지 않을 경우 부정 감성으로 예측

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
  # 감성 지수 초기화
  sentiment=0.0
  tokens_count=0

  lemmatizer = WordNetLemmatizer()
  raw_sentences = sent_tokenize(text)
  # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
  for raw_sentence in raw_sentences:
    # NTLK 기반의 품사 태깅 문장 추출
    tagged_sentence = pos_tag(word_tokenize(raw_sentence))
    for word, tag in tagged_sentence:

      # WordNet 기반 품사 태깅과 어근 추출
      wn_tag = penn_to_wn(tag)
      if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        continue
      lemma = lemmatizer.lemmatize(word, pos=wn_tag)
      if not lemma:
        continue
      # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성.
      synsets = wn.synsets(lemma, pos=wn_tag)
      if not synsets:
        continue
      # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
      # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산.
      synset = synsets[0]
      swn_synset = swn.senti_synset(synset.name())
      sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
      tokens_count += 1
  if not tokens_count:
    return 0

  # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
  if sentiment >= 0:
    return 1
  
  return 0

- swn_polarity(text) 함수를 IMDB 감상평의 개별 문서에 적용해 긍정 및 부정 감성 예측

In [ ]:
# 지도학습 기반의 감성 분석에서 생성한 review_df 그대로 이용
review_df['preds'] = review_df['review'].apply(lambda x : swn_polarity(x))  #새로운 칼럼으로 PREDS를 추가하여 감성 평가를 담아
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

# 감성 분석 예측 성능 
- 정확도, 정밀도, 재현율 값

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np
print(confusion_matrix(y_target, preds))
print("정확도:", np.round(accuracy_score(y_target, preds), 4))
print("정밀도:", np.round(precision_score(y_target, preds), 4))
print("재현율:", np.round(recall_score(y_target, preds), 4))

- 전반적인 성능 평가 지표는 만족스러울 만한 수치는 아님

- SentiWordNet은 WordNet의 하위 모듈로서 감성 분석을 위한 다양한 프레임워크 제공

# VADER를 이용한 감성 분석
- VADER : 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon
- SentimentIntensityAnalyzer 클래스 이용
- NLTK 패키지의 서브 모듈로 제공/ 단독 패키지로 제공

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

- SentimentIntensityAnalyzer 객체 생성 -> 문서별로 polarity_scores() 메서드 호출해 감성 점수 구해 -> 특정 임곗값 이상이면 긍정, 그렇지 않으면 부정으로 판단

- 'neg' 부정 감성 지수, 'neu' 중립적인 감성 지수, 'pos' 긍정 감성 지수

- 'compound'는 neg, neu, pos score를 적절히 조합해 -1에서 1 사이의 감성 지수 표현한 값 -> 보통 0.1 이상이면 긍정 감성, 그 이하이면 부정 감성으로 판단

In [ ]:
def vader_polarity(review, threshold=0.1):
  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(review)

  # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
  agg_score = scores['compound']
  final_sentiment = 1 if agg_score >= threshold else 0
  return final_sentiment

# apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply(lambda x : vader_polarity(x, 0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target, vader_preds), 4))
print("정밀도:", np.round(precision_score(y_target, vader_preds), 4))
print("재현율:", np.round(recall_score(y_target, vader_preds), 4))

- 정확도와 재현율이 SentiWordNet보다 향상
- 이외에도 뛰어난 감성 사전으로 pattern 패키지. but 완벽하게 지원x